<div style="text-align: center; background-color: #559cff; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Lab 02 - Introduction To Data Science @ FIT-HCMUS, VNU-HCM 📌
</div>

<div style="text-align: center; background-color: #b1d1ff; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Stage 02 - Preprocessing
</div>

## Import

In [ ]:
import os
import sys
import json
import ast

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Exploring data

### Combine raw data into dataframes for both video and comment sections

In [ ]:
raw_data_path = '../data/raw/'

with open('../data/external/channel_id_Phuc.json', 'r') as file:
    channel_id = json.load(file)

list_df_videos = []
list_df_comments = []

for channel, id in channel_id.items():
    list_df_videos.append(pd.read_csv(raw_data_path + channel + '_videos.csv', engine = 'python'))
    list_df_comments.append(pd.read_csv(raw_data_path + channel + '_comments.csv', engine = 'python'))
    
df_videos = pd.concat(list_df_videos, ignore_index = True)
df_comments = pd.concat(list_df_comments, ignore_index = True)

In [ ]:
df_videos.head()

In [ ]:
df_comments.head()

### The size of the datasets

#### Videos section

In [ ]:
print('Rows:', df_videos.shape[0])
print('Columns:', df_videos.shape[1])

#### Comments section

In [ ]:
print('Rows:', df_comments.shape[0])
print('Columns:', df_comments.shape[1])

### The meaning of each column

**Videos dataset**
- `playlist_id`: Playlist ID.
- `video_id`: Video ID.
- `title`: Video title.
- `published`: Publication date of the video.
- `view_count`: The video's view count.
- `like_count`: The video's like count.
- `comment_count`: The video's comment count.
- `duration`: How long the video is.
- `definition`: How much information is captured and reproduced (eg. hd, sd).
- `tags`: Tags of the video.
- `default_audio_language`: Default audio language of the video.
- `madeforkid`: Mark if this video is made for kids or not.
- `playlist_title`: Playlist title.
- `channel_id`: The channel ID.
- `channelTitle`: The channel title.
- `playlist_published`: Publication date of the channel.
- `playlist_numvideo`: The number of videos in playlist.

**Comments dataset**
- `Comment_id`: Comment ID.
- `author`: Author of the comment.
- `Reply_for`: The parent comment ID that this comment replied to.
- `Type`: Type of comment (top-level comment (1) or reply(2)).
- `video_id`: The video ID.
- `total_reply`: Total responses to this comment.
- `like_count`: The comment's like count.
- `published_at`: Publication date of the comment.
- `textdisplay`: The content of the comment.
- `updatedat`: The comment's updated date.

### Duplicated rows

#### Videos section

How many duplicated rows do the `df_videos` have?

In [ ]:
df_videos.duplicated().sum()

In [ ]:
df_videos[df_videos.duplicated(keep = False)]

#### Comments section

How many duplicated rows do the `df_comments` have?

In [ ]:
df_comments.duplicated().sum()

In [ ]:
df_comments[df_comments.duplicated(keep = False)]

#### Drop duplicate rows

In [ ]:
df_videos = df_videos.drop_duplicates()
df_comments = df_comments.drop_duplicates()

### Current column types in datasets

#### Videos section

In [ ]:
df_videos.dtypes

The following fields need to be converted to the specified data types:
- `published` and `playlist_published` to datetime.
- `duration` to timedelta.
- `tags` to a list of strings.

In [ ]:
df_videos['published'] =  df_videos['published'].apply(pd.to_datetime)
df_videos['duration'] = df_videos['duration'].apply(pd.to_timedelta)
df_videos['tags'] = df_videos['tags'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else np.nan)
df_videos['playlist_published'] = df_videos['playlist_published'].apply(pd.to_datetime)

#### Comments section

In [ ]:
df_comments.dtypes

The `published_at` and `updatedat` fields need to be converted to datetime.

In [ ]:
df_comments['published_at'] =  df_comments['published_at'].apply(pd.to_datetime)
df_comments['updatedat'] =  df_comments['updatedat'].apply(pd.to_datetime)

Next, we will drop columns that are unnecessary for subsequent steps.

In [ ]:
df_videos = df_videos.drop(columns = ['playlist_id', 'channel_id'])
df_comments = df_comments.drop(columns = ['Comment_id'])

### Distribution of Values for Numeric Columns

In [ ]:
def num_missing_ratio(column):
    return (column.isnull().sum() / len(column) * 100).round(1)

def num_lower_quartile(column):
    return column.quantile(0.25)

def num_median(column):
    return column.quantile(0.5)

def num_upper_quartile(column):
    return column.quantile(0.75)

In [ ]:
num_col_info_df_videos = df_videos.select_dtypes(include = ['int64', 'float64'])

num_col_info_df_videos = num_col_info_df_videos.agg([num_missing_ratio, "min", num_lower_quartile, num_median, num_upper_quartile, "max"])
num_col_info_df_videos

In [ ]:
num_col_info_df_comments = df_comments.select_dtypes(include = ['int64', 'float64'])

num_col_info_df_comments = num_col_info_df_comments.agg([num_missing_ratio, "min", num_lower_quartile, num_median, num_upper_quartile, "max"])
num_col_info_df_comments

### Distribution of Values for Non-numeric Columns

In [ ]:
def cat_missing_ratio(column):
    return (column.isnull().sum() / len(column) * 100).round(1)

def cat_num_values(column):
    if type(column[0] == 'list'):
        column = column.explode()
    return column.nunique()

def cat_value_ratios(column):
    if type(column[0] == 'list'):
        column = column.explode()
    return column.value_counts(normalize=True).sort_values(axis = 0).to_dict()

In [ ]:
cat_col_info_df_videos = df_videos[['definition', 'tags', 'default_audio_language']]

cat_col_info_df_videos = cat_col_info_df_videos.agg([cat_missing_ratio, cat_num_values, cat_value_ratios])
cat_col_info_df_videos

In [ ]:
cat_col_info_df_comments = df_comments[['author']]

cat_col_info_df_comments = cat_col_info_df_comments.agg([cat_missing_ratio, cat_num_values, cat_value_ratios])
cat_col_info_df_comments

### Save the processed data

In [ ]:
processed_data_path = '../data/processed/'

df_videos.to_csv(processed_data_path + 'df_videos_processed.csv', i,,,,ndex = False)
df_comments.to_csv(processed_data_path + 'df_comments_processed.csv', index = False)